# Example queries run on COVID-19 Knowledge Graph on "dsc" Server
[Work in progress]

In [26]:
import os
import time
import pandas as pd
from py2neo import Graph

In [27]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [28]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

### List Organisms in KG

In [29]:
query = """
MATCH (p:Organism)
RETURN p.name as name, p.scientificName as scientificName, p.id as taxonomy
"""
graph.run(query).to_data_frame()

,name,scientificName,taxonomy
0,SARS-CoV-2,Severe acute respiratory syndrome coronavirus 2,taxonomy:2697049
1,MERS-CoV,Middle East respiratory syndrome-related coron...,taxonomy:1335626
2,SARS-CoV,Severe acute respiratory syndrome-related coro...,taxonomy:694009
3,human,Homo sapiens,taxonomy:9606
4,intermediate horseshoe bat,Rhinolophus affinis,taxonomy:59477
5,horseshoe bat,Rhinolophus,taxonomy:49442
6,Malayan pangolin,Manis javanica,taxonomy:9974
7,palm civet,Paradoxurus,taxonomy:71116
8,carnivores,Canidae,taxonomy:9608
9,European mink,Mustela lutreola,taxonomy:9666


### List Coronavirus Outbreaks

In [30]:
query = """
MATCH (p:Organism)-[:CAUSES]->(o:Outbreak)
RETURN p.name as name, p.scientificName as scientificName, p.id as taxonomy, o.id as outbreak, o.startDate as startDate
"""
graph.run(query).to_data_frame()

,name,scientificName,taxonomy,outbreak,startDate
0,SARS-CoV-2,Severe acute respiratory syndrome coronavirus 2,taxonomy:2697049,COVID-19,2019
1,MERS-CoV,Middle East respiratory syndrome-related coron...,taxonomy:1335626,MERS,2012
2,SARS-CoV,Severe acute respiratory syndrome-related coro...,taxonomy:694009,SARS,2003


### List Strains that are mentioned in PubMed Central Articles

In [31]:
query = """
MATCH (p:Publication)-[:MENTIONS]->(s:Strain)<-[:CARRIES]-(o:Organism)
RETURN p.id as pmc, s.name as name, s.collectionDate  as collectionDate, o.name as host, s.id as host_id
ORDER by s.collectionDate
"""
graph.run(query).to_data_frame().head()
# TODO where do the 2013 bat strains come from??

,pmc,name,collectionDate,host,host_id
0,pmc:PMC7166309,bat/Yunnan/RaTG13/2013,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
1,pmc:PMC7095418,bat/Yunnan/RaTG13/2013,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
2,pmc:PMC7166309,bat/Yunnan/RaTG13/2013,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
3,pmc:PMC7166309,bat/Yunnan/RaTG13/2013,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
4,pmc:PMC7095418,bat/Yunnan/RaTG13/2013,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131


### List Gene and Protein information for Reference Genome

In [32]:
query = """
MATCH (s:Strain)-[:HAS]->(g:Gene)-[:ENCODES]->(p:Protein)
RETURN s.id as referenceGenome, s.name as name, s.collectionDate  as collectionDate, 
       g.name as gene, g.id as geneId, p.name as protein, p.id as protein_id 
ORDER by s.collectionDate
"""
graph.run(query).to_data_frame().head()

""


### Cases in a specific County (Admin2)

In [36]:
admin2 = 'San Diego County'

query = """
MATCH (c:Cases{date: date("2020-05-04")})-[:REPORTED_IN]->(a:Admin2{name: $admin2})
RETURN a.name as name, c.cummulativeConfirmed as confirmed, c.cummulativeDeaths as deaths
"""
graph.run(query, admin2=admin2).to_data_frame()

,name,confirmed,deaths
0,San Diego County,3927,139
1,San Diego County,3927,139


### Aggregate cases by State (Admin1)

In [35]:
query = """
MATCH (o:Outbreak{id: "COVID-19"})<-[:RELATED_TO]-(c:Cases{date: date("2020-05-04")})-[:REPORTED_IN]->(a:Admin2)-[:IN]->(a1:Admin1)
RETURN a1.name as state, sum(c.cummulativeConfirmed) as confirmed, sum(c.cummulativeDeaths) as deaths
ORDER BY deaths
"""
graph.run(query).to_data_frame()

,state,confirmed,deaths
0,Rhode Island,68152,0
1,Wyoming,4768,8
2,Alaska,2960,72
3,Montana,3656,128
4,Hawaii,4896,136
5,North Dakota,9800,152
6,South Dakota,21344,168
7,Utah,39352,360
8,West Virginia,9792,384
9,Vermont,7184,416
